In [ ]:
import requests, ast, os
from dotenv import load_dotenv

load_dotenv()

# --- FETCH DETAILS FROM DB ---
import psycopg2

DB_CONFIG = {
                "host": "<insert_value_here>",
                "port": "<insert_value_here>",
                "dbname": "<insert_value_here>",
                "user": "<insert_value_here>",
                "password": "<insert_value_here>SbMV0p8LWqYl"
        }
        
connection_uri = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

ref_num = "221234COE23022024113947"
query = f"SELECT emp_email, emp_id, details, approval_status FROM procedural_data.coe_requests_v2 where reference_id = '{ref_num}';"

conn = psycopg2.connect(connection_uri)
cur = conn.cursor()
cur.execute(query)
email_raw = cur.fetchall()

conn.commit()
conn.close()

date_dict = ast.literal_eval(email_raw[0][2])
departure_date, arrival_date = date_dict["departure_date"], date_dict["arrival_date"]
# email = email_raw[0][0]
emp_id = email_raw[0][1]
email = "eferrancullo@77global.biz"
status = email_raw[0][3]

# ---- EMAIL SERVICE ---
email_service_api = os.getenv("unsec_endpoint")

headers = {
    'Accept':'*/*',
    'Content-Type': 'application/json'
}

html_body = f"""<html>
    <div class="container" style="font-family: monospace; font-size:1.18rem;">
      <p style="font-size:1.25rem;">Good day!</p>
      <p>
        COE request with the following details has been filed:<br><br>
        Ref Number: {ref_num}<br>
        Employee Id: {emp_id}<br>
        Employee email: {email}<br>
        Departure Date: {departure_date}<br>
        Arrival Date: {arrival_date}<br>
        Status: {status}
      </p>

      <form action="https://063qgljmkj.execute-api.us-east-1.amazonaws.com/default/chatbot_post_receiver" method="POST">
                <button name="ref_num" value="{ref_num}">Approve</button>
            </form>
      
      <br><small><i>This is a system generated email, please do not reply</i></small>
    </div>
</html>"""

data = {
    "emailBcc":"",
    "emailBody":html_body,
    "emailCc":"ndevilla@77global.biz; bvillamil@77global.biz",
    "emailFrom":"no-reply-alicia@77soft.com",
    "emailSubject":"HTML Email Test Using Python",
    "emailTo":"eferrancullo@77global.biz"
}

response = requests.post(email_service_api, headers=headers, json=data)
print(response.json())